# Table of Content

# Imports Always First

In [1]:
import pandas as pd
import openpyxl as op
import os

## <li id="cd">Creating DataFrames

### <ol> Transactions

In [2]:
transactions = {'TranDate': ['2017-01-01'],
                'TranDetail': ['The Local'],
                'TranStatus': ['Restaurant'],
                'TranAmount': [-10.00],
                'Balance': [10.00]}
transactions_df = pd.DataFrame(transactions).assign(Category = None)
transactions_df

,TranDate,TranDetail,TranStatus,TranAmount,Balance,Category
0,2017-01-01,The Local,Restaurant,-10.0,10.0,None


### <ol> Categories

In [3]:
categories = ['Groceries', 'Restaurant', 'Entertainment']
categories_df = pd.DataFrame(categories, columns=['Categories'])
categories_df

,Categories
0,Groceries
1,Restaurant
2,Entertainment


# Excel Writer

## <li id=""> Creating File
    This is a way to create the excel file using the Workbook object. It will create the xlsx file with one sheet named 'Sheet'.

In [9]:
wb = op.Workbook()
wb.save('example_writer.xlsx')
print(wb.sheetnames)

['Sheet']


## <li id=""> Insert Transactions Dataframe in First Sheet
    To add the transactions table to the sheet, we can use the to_excel method. However, this will just replace the existing sheets in the file.

In [10]:
# This will override the above cell
# Thus you can create a new workbook using this method
transactions_df.to_excel('example_writer.xlsx', sheet_name='ChangeMyName')

## <li id=""> Loading Workbook 
    As we can see, the sheet name 'Sheet' was replaced with 'ChangeMyName' sheet. This is because the to_excel method just replaced the workbook instead of just adding the new sheet.

In [11]:
book = op.load_workbook('example_writer.xlsx')
print(book.sheetnames)

['ChangeMyName']


## <li id=""> Change Sheet Name
    We can change the name of the sheet by slicing on the sheetname and changing the title attribute name.

In [12]:
sheet = book['ChangeMyName']
sheet.title = 'Transactions'
book.sheetnames

['Transactions']

## <li id=""> Create Pandas Writer
    To avoid the workbook from being replaced without the existing sheets, we use the book from which we loaded the Excel file. That is opxl.load_workbook(). This object will contains all the existing sheets. We create a new writer using the ExcelWriter object in pandas and assign the book and the sheets to the writer.

In [13]:
# Creating the writer
writer = pd.ExcelWriter('example_writer.xlsx', engine='openpyxl')
# Assigning the opened workbook as the writer book
writer.book = book
# Copying all the sheets from the current workbook
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

## <li id=""> Insert Category Dataframe in Second Sheet

In [14]:
categories_df.to_excel(writer, 'Categories')
print(book.sheetnames)

['Transactions', 'Categories']


### <ol> A. Creating Data Validation out of Categories

#### <ol><ol> 1. Extracting Cell Reference

In [81]:
category_count = categories_df.shape[0]
column_idx = transactions_df.T.reset_index()['index'].loc[lambda x: x=='Category'].index[0] + 1 
column_letter = chr(65+column_idx)
row_start = 2
row_end = 1 + category_count
formula_range = f"{column_letter}{row_start}:{column_letter}{row_end}"
formula_range

'G2:G5'

#### <ol><ol> 2. Creating Data Validations

In [88]:
# Getting names to create formula and add dv to sheet
writer.book.sheetnames

['Transactions', 'Categories']

In [90]:
dv = op.worksheet.datavalidation.DataValidation(type='list', formula1=f"=Categories!B{row_start}:B{row_end}")
ws_tran = writer.book['Transactions']
ws_tran.add_data_validation(dv)
dv.add(formula_range)

## <li> Changing Width of Columns

### <ol> A. Iterating Over Populated Columns

In [106]:
# Populated Columns
populated_columns = [x[0].column for x in ws_tran.columns]
populated_columns

['A', 'B', 'C', 'D', 'E', 'F', 'G']

### <ol> B. Getting Sizes of Columns by Using String Lenghts

In [166]:
min_size = max([len(col) for col in transactions_df.columns.tolist()]) * 1.145
min_size 

11.45

In [168]:
column_sizes = transactions_df.reset_index().apply(lambda x: x.apply(lambda x: len(str(x)))).max().tolist()
print(column_sizes)
column_sizes = [size if size >= min_size else min_size for size in column_sizes]
column_sizes

[1, 10, 9, 10, 5, 4, 4]


[11.45, 11.45, 11.45, 11.45, 11.45, 11.45, 11.45]

### <ol> C. Applying Column Widths to Sheet

In [162]:
zipped_list = list(zip(populated_columns, tran_sizes))
zipped_list

[('A', 11.45),
 ('B', 11.45),
 ('C', 11.45),
 ('D', 11.45),
 ('E', 11.45),
 ('F', 11.45),
 ('G', 11.45)]

In [163]:
for column, size in zipped_list:
    ws_tran.column_dimensions[column].width = size

In [15]:
writer.save()
os.startfile('example_writer.xlsx')

In [170]:
categories_df.index.name='index'

In [173]:
categories_df.reset_index(drop=True)

,Categories
0,Groceries
1,Restaurant
2,Entertainment
